In [1]:
import tensorflow as tf
import numpy as np

# 定义超参数
learning_rate = 0.001
training_steps = 10000
batch_size = 32
display_step = 200

# 加载数据
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train/255., x_test/255.

# 将数据转换成序列的形式
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

# 定义网络结构
class RNN(tf.keras.Model):
    def __init__(self, num_units, num_classes):
        super(RNN, self).__init__()
        self.num_units = num_units
        self.num_classes = num_classes
        self.rnn_layer = tf.keras.layers.LSTM(self.num_units)
        self.fc_layer = tf.keras.layers.Dense(self.num_classes)
        
    def call(self, x):
        # RNN layer
        x = self.rnn_layer(x)
        # fully connected layer
        x = self.fc_layer(x)
        return x

# 定义模型
model = RNN(num_units=64, num_classes=10)

# 定义损失函数和优化器
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 定义性能指标
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 训练模型
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # 将数据转换成可训练模型输入的格式
    batch_x = tf.reshape(batch_x, shape=[-1, 28, 28])
    # 计算梯度
    with tf.GradientTape() as tape:
        predictions = model(batch_x)
        loss = loss_object(batch_y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # 记录性能指标
    train_loss(loss)
    train_accuracy(batch_y, predictions)
    # 显示训练进度
    if step % display_step == 0:
        print("step: %i, loss: %f, accuracy: %f" % (step, train_loss.result(), train_accuracy.result()))


11490434/11490434 [==============================] - 5s 0us/step
step: 200, loss: 3.727396, accuracy: 0.146406
step: 400, loss: 3.015347, accuracy: 0.140078
step: 600, loss: 2.778035, accuracy: 0.143281
step: 800, loss: 2.659399, accuracy: 0.144805
step: 1000, loss: 2.588017, accuracy: 0.145781
step: 1200, loss: 2.540430, accuracy: 0.146016
step: 1400, loss: 2.506387, accuracy: 0.144888
step: 1600, loss: 2.480855, accuracy: 0.144785
step: 1800, loss: 2.464123, accuracy: 0.144740
step: 2000, loss: 2.447985, accuracy: 0.145047
step: 2200, loss: 2.434780, accuracy: 0.146463
step: 2400, loss: 2.423776, accuracy: 0.146042
step: 2600, loss: 2.414465, accuracy: 0.146334
step: 2800, loss: 2.406485, accuracy: 0.146529
step: 3000, loss: 2.399568, accuracy: 0.147146
step: 3200, loss: 2.393516, accuracy: 0.147861
step: 3400, loss: 2.388176, accuracy: 0.148116
step: 3600, loss: 2.383429, accuracy: 0.148125
step: 3800, loss: 2.379182, accuracy: 0.148372
step: 4000, loss: 2.375359, accuracy: 0.148578

In [2]:
from torch.utils.tensorboard import SummaryWriter
import shutil
import os
import random

dir_path = './logs'

try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))

if not os.path.exists("./logs"):
    os.makedirs("./logs")

writer = SummaryWriter('./logs')

for epoch in range(100):
    train_loss = epoch**2+3*epoch + 5*random.uniform(0,100)
    writer.add_scalar("loss/train", train_loss, epoch)

writer.close()


Error: ./logs : 系统找不到指定的路径。
